# Project 3

### University of California, Berkeley
### Master of Information and Data Science Program (MIDS)
### w205 - Fundamentals of Data Engineering

* Year: 2022
* Semester: Summer
* Section: 02
* Instructor: Korin Reid
* Team Members:
    * team member 1 Iris Lew
    * team member 2 Ivy Chan
    * team member 3 Ghiwa Lamah


# Install and Import the necessary packages

In [1]:
!pip install folium

     |████████████████████████████████| 95 kB 6.8 MB/s  eta 0:00:01


In [2]:
import neo4j

import csv
import json

import math
import numpy as np
import pandas as pd

import psycopg2

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

from IPython.display import display

import folium
from folium.plugins import HeatMap

AGM's vision for the future includes partnering with BART for one of the several options:

* Adding more pickup locations
* Using BART to transport the deliveries
* Using delivery drones or robots (delivery range of 1 mile)
* a Hybrid version where BART is used as pickup and transport, and then the delivery drones or robots can take them to people's homes.

This can be accomplished using Neo4j and Postgres.

In [3]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [4]:
session = driver.session(database="neo4j")

In [5]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [6]:
cursor = connection.cursor()

These are useful functions. That we will use throughout the project.

In [7]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [8]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [9]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [10]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [11]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [12]:
def my_neo4j_create_node(station_name, name_label):
    "create a node with label Station"
    
    query = """ CREATE (:"""
    query += name_label
    query += """ {name: $station_name})"""
    
    
    session.run(query, station_name=station_name)

In [13]:
def my_neo4j_create_relationship_one_way(from_station, to_station, to, weight, rel_type):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:"""
    query += to
    query += """)
    
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:""" + rel_type
    
    query += " {weight: $weight}]->(to)"
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [14]:
# def my_neo4j_create_relationship_two_way(from_station, to_station, to, weight):
#     "create relationships two way between two stations with a weight"
    
#     query = """
    
#     MATCH (from:Station), 
#           (to:"""
#     query += to
#     query += """)
    
#     WHERE from.name = $from_station and to.name = $to_station
#     CREATE (from)-[:LINK {weight: $weight}]->(to),
#            (to)-[:LINK {weight: $weight}]->(from)
    
#     """
    
#     session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [15]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

In [16]:
def my_pickuplocat_get_zips(pickuplocat, miles,customer=False):
    "given a pickup locatation, pull all zip codes with miles distance, print them, sum the population and customers"
    
    connection.rollback()
    
    query = "select latitude, longitude from "
    query += "(select station,latitude, longitude from stations "
    query += "union select concat(city,' store') as station, latitude, longitude from stores) locat_gps "
    query += "where station = '" + pickuplocat + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select z.zip, z.population "
    if customer == True:
        query += " , count(c.customer_id) as cnt "
    query += " from zip_codes as z "
    if customer == True:
        query += " join customers as c on z.zip=c.zip "
    query += " where z.latitude >= " + str(bottom[0])
    query += " and z.latitude <= " + str(top [0])
    query += " and z.longitude >= " + str(left[1])
    query += " and z.longitude <= " + str(right[1])
    if customer == True:
        query += " group by z.zip,z.population "
    query += " order by 1 "
    
    

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    total_population = 0
    if customer==True:
        total_customers = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        if customer==False:
            pass
        if customer==True:
            customers = row[2]
            total_customers += customers
        
        total_population += population
        
    if customer:
        return total_customers
    else:
        return total_population


We plan to use Neo4j to figure out which are the shortest path for optimal delivery time using BART as transportation. Afterwards, we will examine which stations will serve as the best pickup locations: stations where there is a high population but few customers, or the most traffic. The stations with the most traffic, using the location where many of the local customers reside as a proxy, can serve as tests for drone and robot delivery.

# BART Delivery: Verify Shortest Paths

We can use BART to transport the deliveries, and from there, customers can pickup their meals or the drones or robots can take them to people's homes. Thus, we first needed to verify the shortest path from one station to another.

In [17]:
my_neo4j_wipe_out_database()

In [18]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

## Use the function my_neo4j_shortest_path() to check and compare the travel times between Downtown Berkeley to the end stations of all lines. Our main store is located at Berkeley.

In [19]:
# Richmond-Millbrae+SFO Line (Red) end station 
# Berryessa/North San Jose-Richmond Line (orange) end station
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive Richmond')

In [20]:
# Antioch-SFO+Millbrae Line (Yellow) end station 
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive Antioch')

In [21]:
# Richmond-Millbrae+SFO Line (Red) end station 
# Antioch-SFO+Millbrae Line (Yellow) end station 
# Dublic/Pleasanton-Daly City Line (Blue) end station
# Berryessa/North San Jose-Daly City Line (Green) end station
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive Daly City')

In [22]:
# Richmond-Millbrae+SFO Line (Red) end station 
# Antioch-SFO+Millbrae Line (Yellow) end station 
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive Millbrae')

In [23]:
# Dublic/Pleasanton-Daly City Line (Blue) end station
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive Dublin')

In [24]:
# Berryessa/North San Jose-Daly City Line (Green) end station
# Berryessa/North San Jose-Richmond Line (orange) end station
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive Berryessa')

In [25]:
# Oakland International Airport (Gray)
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive OAK')

In [26]:
# San Francisco International Airport (Red & Yellow)
my_neo4j_shortest_path('depart Downtown Berkeley', 'arrive SFO')

## Analysis on Shortest Paths from the Downtown Berkeley Station to all end stations of all BART lines.

We applied Dijkstra's algorithm on Neo4j to identify the shortest paths from the Downtown Berkeley station to the end stations of all BART lines. The end stations include Richmond, Antioch, Daly City, Millbrae, Dublin, Berryessa, Oakland Airport, and San Francisco Airport. The commute time ranges from 13 minutes to 70 minutes.

The shortest trip time is from the Downtown Berkeley Station to the Richmond Station, which takes 13 minutes with either the red or the orange line. The most extended trip is from the Downtown Berkeley Station to the Berryessa Station, which takes 70 minutes or 1 hour and 10 minutes with the orange line. Coincidently, both trips do not need to transfer at any transferring stations.

For the trips require the transfer, such as the trips from the Downtown Berkeley station to the Antoich, Dublin, Oakland Airport, and San Francisco Airport stations, we transfer at the MacArthur, Coliseum, and Balboa Park stations.

Obviously, the closer to the Downtown Berkeley Station, the shorter the commute time.



# Pickup Locations

We want to examine which BART stations can serve as the best pickup locations. These are the ones where there is any combination of
* high amount of traffic
* high population
* low customers
* far from store

In [27]:
my_neo4j_wipe_out_database()

In [28]:
### For Graph Database relationships

## Query all scheduled trains going from one BART station to another and save in dataframe

rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

bart_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [29]:
# Get stations
connection.rollback()
    
query = "select station from stations "

stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [30]:
# get coordinates of Berkeley store
connection.rollback()
    
query = "select latitude, longitude from stores where city = 'Berkeley'  "

cursor.execute(query)

connection.rollback()

coordinates = cursor.fetchall()
lat = float(coordinates[0][0])
lon = float(coordinates[0][1])

In [31]:
def my_calculate_distance(station, store):
    "calculate the distance between the station and the Berkeley store"
    
    geod = Geodesic.WGS84

    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)

    g = geod.Inverse(float(point[0]), float(point[1]), store[0], store[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

In [32]:
df = pd.DataFrame(columns = ['Station', 'Population_Within_1_Mile', 'Customers_Within_1_Mile', 'Distance_From_Store'])
df = df.append({'Station' : 'Berkeley Store', 'Population_Within_1_Mile' : 0, 
                    'Customers_Within_1_Mile': 0, 'Distance_From_Store': 0},
                   ignore_index = True)
for station in list(stations.station):
    total_population = my_pickuplocat_get_zips(station, 1, False)
    total_customers = my_pickuplocat_get_zips(station, 1, True)
    distance = my_calculate_distance(station, (lat, lon))
    if total_population > 0:
        df = df.append({'Station' : station, 'Population_Within_1_Mile' : total_population, 
                    'Customers_Within_1_Mile': total_customers, 'Distance_From_Store': round(distance,1)},
                   ignore_index = True)
df.sort_values('Population_Within_1_Mile', ascending = False)

,Station,Population_Within_1_Mile,Customers_Within_1_Mile,Distance_From_Store
29,Powell Street,140730,333,9.5
17,Glen Park,115068,103,12.7
4,24th Street Mission,108915,133,11.2
6,Balboa Park,106589,76,13.8
27,Pittsburg Center,96081,1,23.1
23,Montgomery Street,85465,274,9.0
11,Downtown Berkeley,82930,634,1.1
34,South Hayward,79235,9,18.9
9,Civic Center,74898,210,9.9
35,Union City,74601,9,22.6


In [33]:
for index, row in df.iterrows():
    station = row[0]
    num_population = float(row[1])
    num_customers = float(row[2])
    distance_from_store = row[3]
    ratio_dp_c = distance_from_store * num_population / (num_customers + 0.1)    
    
    my_neo4j_create_node(station, "Station")
    if index > 0:
        my_neo4j_create_relationship_one_way(prev_station, station, "Station", ratio_dp_c, "LINK")
    
    prev_station = row[0]
    
for index, row in bart_df.iterrows():
    from_station = row[1]
    to_station = row[2]
    my_neo4j_create_relationship_one_way(from_station, to_station, "Station", 0, "LINK")

## Weighted Degree Centrality

In [34]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.create('ds_graph', 'Station','LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [35]:
query = """

CALL gds.degree.stream(
   'ds_graph',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS Station, score AS Degree_Centrality
ORDER BY Degree_Centrality DESC, Station DESC
"""

my_neo4j_run_query_pandas(query)

,Station,Degree_Centrality
0,Bay Fair,1.140422e+07
1,Orinda,2.017701e+06
2,Civic Center,2.579654e+05
3,Embarcadero,2.352419e+05
4,South Hayward,1.852728e+05
5,San Leandro,1.645650e+05
6,Downtown Berkeley,1.206939e+05
7,Glen Park,1.039891e+05
8,SFO,8.150910e+04
9,MacArthur,4.249693e+04


## Page Rank (Trials)

In [36]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor,
                           relationshipWeightProperty: 'weight'}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS Station, score as Page_Rank
ORDER BY Page_Rank DESC, Station ASC

"""

max_iterations = 20
damping_factor = 0.05

# df_pr = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)[1:]

my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)[1:]



,Station,Page_Rank
1,Bay Fair,1.000000
2,Berryessa,1.000000
3,Civic Center,1.000000
4,Colma,1.000000
5,Downtown Berkeley,1.000000
6,Dublin,1.000000
7,El Cerrito Plaza,1.000000
8,Embarcadero,1.000000
9,Fremont,1.000000
10,Fruitvale,1.000000


In [37]:
query = """

MATCH (siteA:Station {name: $source})
CALL gds.pageRank.stream('ds_graph', {
  maxIterations: $max_iterations,
  dampingFactor: $damping_factor,
  sourceNodes: [siteA],
  relationshipWeightProperty: 'weight'
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY score DESC, name ASC

"""

source = "Berryessa"
max_iterations = 20
damping_factor = 0.85

my_neo4j_run_query_pandas(query, source=source, max_iterations=max_iterations, damping_factor=damping_factor)


,name,page_rank
0,Berryessa,0.150000
1,Civic Center,0.127500
2,Colma,0.108375
3,Downtown Berkeley,0.092119
4,Dublin,0.078301
5,El Cerrito Plaza,0.066556
6,Embarcadero,0.056572
7,Fremont,0.048087
8,Fruitvale,0.040874
9,Glen Park,0.034743


# Drone and Robots Delivery

Drone and robot delivery is an investment. It costs money to buy the drone or robot, charge it, and then program it to the customers' homes and return so it would be in AGM's best interests to test if there is an appetite for this delivery option. Furthermore drone and robots have a limited range of 1 mile. Thus, we want to determine which BART stations have the most customers who reside within 1 mile of the stations using the Page Rank algorithm. The Page Rank algorithm determines which nodes, and thus stations, are the most influential through looking at the incoming relationships. 

In [38]:
my_neo4j_wipe_out_database()

In [39]:
connection.rollback()

station_list = []
pop_counts = []
cust_1mi_counts = []

query = "select latitude, longitude, station from "
query += "(select station,latitude, longitude from stations "
query += "union select concat(city,' store') as station, latitude, longitude from stores where city = 'Berkeley') locat_gps"

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    latitude = row[0]
    longitude = row[1]
    station = row[2]
    station_list.append(station)
    my_neo4j_create_node(station, "Station")

    point = (latitude, longitude)

    (left, right, top, bottom) = my_calculate_box(point, 1)

    query = "select z.zip, z.population, count(c.customer_id) as cnt "
    query += " from zip_codes as z "
    query += " join customers as c on z.zip=c.zip "
    query += " where z.latitude >= " + str(bottom[0])
    query += " and z.latitude <= " + str(top [0])
    query += " and z.longitude >= " + str(left[1])
    query += " and z.longitude <= " + str(right[1])
    query += " group by z.zip,z.population "
    query += " order by 1 "

    cursor.execute(query)

    connection.rollback()

    rows = cursor.fetchall()


    total_population = 0
    total_customers = 0

    for row in rows:
        zip_code = row[0]
        population = row[1]
        customers = row[2]
        total_customers += customers
        total_population += population
    pop_counts.append(int(total_population))
    cust_1mi_counts.append(total_customers)
        

In [40]:
stations_cust = pd.DataFrame(list(zip(station_list, pop_counts, cust_1mi_counts)), 
                             columns = ["Station", "Population", "Customers"])
stations_cust.sort_values(by="Station")

,Station,Population,Customers
49,12th Street,16062,161
23,16th Street Mission,63489,135
31,19th Street,16062,161
16,24th Street Mission,108915,133
11,Antioch,0,0
22,Ashby,68219,487
14,Balboa Park,106589,76
20,Bay Fair,41059,63
21,Berkeley store,51127,316
47,Berryessa,0,0


In [41]:
my_neo4j_create_relationship_one_way("Antioch","Pittsburg Center","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Pittsburg Center","Pittsburg","Station",1, "LINK")
my_neo4j_create_relationship_one_way("Pittsburg Center","Antioch","Station",1,"LINK")

my_neo4j_create_relationship_one_way("Pittsburg","Pittsburg Center","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Pittsburg","North Concord","Station",0,"LINK")

my_neo4j_create_relationship_one_way("North Concord","Pittsburg","Station",35,"LINK")
my_neo4j_create_relationship_one_way("North Concord","Concord","Station",35,"LINK")

my_neo4j_create_relationship_one_way("Concord","North Concord","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Concord","Pleasant Hill","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Pleasant Hill","Concord","Station",74,"LINK")
my_neo4j_create_relationship_one_way("Pleasant Hill","Walnut Creek","Station",74,"LINK")

my_neo4j_create_relationship_one_way("Walnut Creek","Pleasant Hill","Station",74,"LINK")
my_neo4j_create_relationship_one_way("Walnut Creek","Lafayette","Station",74,"LINK")

my_neo4j_create_relationship_one_way("Lafayette","Walnut Creek","Station",129,"LINK")
my_neo4j_create_relationship_one_way("Lafayette","Orinda","Station",129,"LINK")

my_neo4j_create_relationship_one_way("Orinda","Lafayette","Station",193,"LINK")
my_neo4j_create_relationship_one_way("Orinda","Rockridge","Station",193,"LINK")

my_neo4j_create_relationship_one_way("Rockridge","Orinda","Station",311,"LINK")
my_neo4j_create_relationship_one_way("Rockridge","MacArthur","Station",311,"LINK")

my_neo4j_create_relationship_one_way("MacArthur","Rockridge","Station",155,"LINK")
my_neo4j_create_relationship_one_way("MacArthur","19th Street","Station",155,"LINK")
my_neo4j_create_relationship_one_way("MacArthur","Ashby","Station",155,"LINK")

my_neo4j_create_relationship_one_way("19th Street","MacArthur","Station",161,"LINK")
my_neo4j_create_relationship_one_way("19th Street","12th Street","Station",161,"LINK")

my_neo4j_create_relationship_one_way("12th Street","19th Street","Station",161,"LINK")
my_neo4j_create_relationship_one_way("12th Street","Lake Merritt","Station",161,"LINK")
my_neo4j_create_relationship_one_way("12th Street","West Oakland","Station",161,"LINK")

my_neo4j_create_relationship_one_way("Lake Merritt","12th Street","Station",161,"LINK")
my_neo4j_create_relationship_one_way("Lake Merritt","Fruitvale","Station",161,"LINK")
my_neo4j_create_relationship_one_way("Lake Merritt","West Oakland","Station",161,"LINK")

my_neo4j_create_relationship_one_way("Fruitvale","Lake Merritt","Station",180,"LINK")
my_neo4j_create_relationship_one_way("Fruitvale","Coliseum","Station",180,"LINK")

my_neo4j_create_relationship_one_way("Coliseum","Fruitvale","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Coliseum","OAK","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Coliseum","San Leandro","Station",0,"LINK")

my_neo4j_create_relationship_one_way("OAK","Coliseum","Station",0,"LINK")

my_neo4j_create_relationship_one_way("San Leandro","Coliseum","Station",72,"LINK")
my_neo4j_create_relationship_one_way("San Leandro","Bay Fair","Station",72,"LINK")

my_neo4j_create_relationship_one_way("Bay Fair","San Leandro","Station",63,"LINK")
my_neo4j_create_relationship_one_way("Bay Fair","Castro Valley","Station",63,"LINK")
my_neo4j_create_relationship_one_way("Bay Fair","Hayward","Station",63,"LINK")

my_neo4j_create_relationship_one_way("Castro Valley","Bay Fair","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Castro Valley","West Dublin","Station",0,"LINK")

my_neo4j_create_relationship_one_way("West Dublin","Dublin","Station",0,"LINK")
my_neo4j_create_relationship_one_way("West Dublin","Castro Valley","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Dublin","West Dublin","Station",10,"LINK")

my_neo4j_create_relationship_one_way("Hayward","Bay Fair","Station",10,"LINK")
my_neo4j_create_relationship_one_way("Hayward","South Hayward","Station",10,"LINK")

my_neo4j_create_relationship_one_way("South Hayward","Hayward","Station",9,"LINK")
my_neo4j_create_relationship_one_way("South Hayward","Union City","Station",9,"LINK")

my_neo4j_create_relationship_one_way("Union City","South Hayward","Station",9,"LINK")
my_neo4j_create_relationship_one_way("Union City","Fremont","Station",9,"LINK")

my_neo4j_create_relationship_one_way("Fremont","Union City","Station",8,"LINK")
my_neo4j_create_relationship_one_way("Fremont","Warm Springs","Station",8,"LINK")

my_neo4j_create_relationship_one_way("Warm Springs","Fremont","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Warm Springs","Milpitas","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Milpitas","Warm Springs","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Milpitas","Berryessa","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Berryessa","Milpitas","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Ashby","MacArthur","Station",487,"LINK")
my_neo4j_create_relationship_one_way("Ashby","Downtown Berkeley","Station",487,"LINK")

my_neo4j_create_relationship_one_way("Downtown Berkeley","Ashby","Station",634,"LINK")
my_neo4j_create_relationship_one_way("Downtown Berkeley","North Berkeley","Station",634,"LINK")

my_neo4j_create_relationship_one_way("North Berkeley","Downtown Berkeley","Station",443,"LINK")
my_neo4j_create_relationship_one_way("North Berkeley","El Cerrito Plaza","Station",443,"LINK")

my_neo4j_create_relationship_one_way("El Cerrito Plaza","El Cerrito del Norte","Station",175,"LINK")
my_neo4j_create_relationship_one_way("El Cerrito Plaza","North Berkeley","Station",175,"LINK")

my_neo4j_create_relationship_one_way("El Cerrito del Norte","El Cerrito Plaza","Station",0,"LINK")
my_neo4j_create_relationship_one_way("El Cerrito del Norte","Richmond","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Richmond","El Cerrito del Norte","Station",0,"LINK")

my_neo4j_create_relationship_one_way("West Oakland","12th Street","Station",203,"LINK")
my_neo4j_create_relationship_one_way("West Oakland","Lake Merritt","Station",203,"LINK")
my_neo4j_create_relationship_one_way("West Oakland","Embarcadero","Station",203,"LINK")

my_neo4j_create_relationship_one_way("Embarcadero","West Oakland","Station",196,"LINK")
my_neo4j_create_relationship_one_way("Embarcadero","Montgomery Street","Station",196,"LINK")

my_neo4j_create_relationship_one_way("Montgomery Street","Embarcadero","Station",274,"LINK")
my_neo4j_create_relationship_one_way("Montgomery Street","Powell Street","Station",274,"LINK")

my_neo4j_create_relationship_one_way("Powell Street","Montgomery Street","Station",333,"LINK")
my_neo4j_create_relationship_one_way("Powell Street","Civic Center","Station",333,"LINK")

my_neo4j_create_relationship_one_way("Civic Center","Powell Street","Station",210,"LINK")
my_neo4j_create_relationship_one_way("Civic Center","16th Street Mission","Station",210,"LINK")

my_neo4j_create_relationship_one_way("16th Street Mission","Civic Center","Station",135,"LINK")
my_neo4j_create_relationship_one_way("16th Street Mission","24th Street Mission","Station",135,"LINK")

my_neo4j_create_relationship_one_way("24th Street Mission","16th Street Mission","Station",133,"LINK")
my_neo4j_create_relationship_one_way("24th Street Mission","Glen Park","Station",133,"LINK")

my_neo4j_create_relationship_one_way("Glen Park","24th Street Mission","Station",103,"LINK")
my_neo4j_create_relationship_one_way("Glen Park","Balboa Park","Station",103,"LINK")

my_neo4j_create_relationship_one_way("Balboa Park","Glen Park","Station",76,"LINK")
my_neo4j_create_relationship_one_way("Balboa Park","Daly City","Station",76,"LINK")

my_neo4j_create_relationship_one_way("Daly City","Balboa Park","Station",0,"LINK")
my_neo4j_create_relationship_one_way("Daly City","Colma","Station",0,"LINK")

my_neo4j_create_relationship_one_way("Colma","Daly City","Station",4,"LINK")
my_neo4j_create_relationship_one_way("Colma","South San Francisco","Station",4,"LINK")

my_neo4j_create_relationship_one_way("South San Francisco","Colma","Station",0,"LINK")
my_neo4j_create_relationship_one_way("South San Francisco","San Bruno","Station",0,"LINK")

my_neo4j_create_relationship_one_way("San Bruno","South San Francisco","Station",9,"LINK")
my_neo4j_create_relationship_one_way("San Bruno","SFO","Station",9,"LINK")
my_neo4j_create_relationship_one_way("San Bruno","Millbrae","Station",9,"LINK")

my_neo4j_create_relationship_one_way("SFO","San Bruno","Station",1,"LINK")
my_neo4j_create_relationship_one_way("SFO","Millbrae","Station",1,"LINK")

my_neo4j_create_relationship_one_way("Millbrae","San Bruno","Station",10,"LINK")
my_neo4j_create_relationship_one_way("Millbrae","SFO","Station",10,"LINK")


In [42]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 51
  Relationships: 102
-------------------------


In [43]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [44]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,12th Street,[Station]
1,16th Street Mission,[Station]
2,19th Street,[Station]
3,24th Street Mission,[Station]
4,Antioch,[Station]
5,Ashby,[Station]
6,Balboa Park,[Station]
7,Bay Fair,[Station]
8,Berkeley store,[Station]
9,Berryessa,[Station]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,12th Street,[Station],LINK,19th Street,[Station]
1,12th Street,[Station],LINK,Lake Merritt,[Station]
2,12th Street,[Station],LINK,West Oakland,[Station]
3,16th Street Mission,[Station],LINK,24th Street Mission,[Station]
4,16th Street Mission,[Station],LINK,Civic Center,[Station]
...,...,...,...,...,...
97,West Dublin,[Station],LINK,Castro Valley,[Station]
98,West Dublin,[Station],LINK,Dublin,[Station]
99,West Oakland,[Station],LINK,12th Street,[Station]
100,West Oakland,[Station],LINK,Embarcadero,[Station]


-------------------------
  Density: 0.1
-------------------------


## Page Rank

We use the Page Rank algorithm to determine which stations have the most nearby customers for delivery robots and drones.

In [45]:
query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor,
                           relationshipWeightProperty: 'weight'}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS station, score as page_rank
ORDER BY page_rank DESC, station ASC

"""




max_iterations = 20
damping_factor = 0.05

my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)

,station,page_rank
0,MacArthur,1.024200
1,West Oakland,1.008395
2,12th Street,1.008200
3,Lake Merritt,1.007780
4,Civic Center,1.000000
5,16th Street Mission,1.000000
6,Powell Street,0.999995
7,24th Street Mission,0.999984
8,Lafayette,0.999979
9,Orinda,0.999801


We find that the top three BART stations using the Page Rank Algorithm are the MacArthur, West Oakland, and 12th Street Stations. Most customers reside around Berkeley and North San Francisco so this matches with our intuition.

# Hybrid

If AGM wishes, we can combine what we know from Dijkstra's algorithm to find the shortest transportation path using BART, and then have one of two options:
1. have the customers pick up using BART
2. use a drone, robot, or truck deliver the food to the customers' homes

The traditional truck can deliver to multiple homes while a drone or robot will likely only deliver to one home. We can use the same information from the Page Rank algorithm, but will have to assess demand before allocating resources to invest in delivery trucks.

# Map Visualizations

## BART on a Map
We want to create a map that shows where the Berkeley store is and where the BART goes.
1. Connect Google Map
2. Get the Berkeley Store latidude and longitude
3. Create the map
4. Add a transit layer

In [46]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [47]:
berkeley_store = (lat, lon)
berkeley_store

(37.8555, -122.2604)

In [48]:
fig = gmaps.figure(center=berkeley_store, zoom_level=10)

fig.add_layer(gmaps.transit_layer())

fig

Figure(layout=FigureLayout(height='420px'))

## Customer Locations

We want to plot the BART stations on the map and figure out which areas have the highest density of customers who live within 1 mile of the station.

1. Get the GPS coordinates of the stations
2. Get the list of zip codes that are within 1 mile of the BART station or Berkeley store
3. Query for the customers who reside in those zip codes
4. Plot on a map
5. Add markers for where the stations are on the map. We used this resource to guide us on customizing the markers: https://www.python-graph-gallery.com/312-add-markers-on-folium-map

In [49]:
zips_1mi = []
lat_1mi = []
lon_1mi = []


connection.rollback()

query = "select latitude, longitude from "
query += "(select station,latitude, longitude from stations "
query += "union select concat(city,' store') as station, latitude, longitude from stores) locat_gps "

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()


for i in rows:
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]

        point = (latitude, longitude)

        (left, right, top, bottom) = my_calculate_box(point, 1)

        query = "select z.zip, z.latitude, z.longitude"
        query += " from zip_codes as z "
        query += " where z.latitude >= " + str(bottom[0])
        query += " and z.latitude <= " + str(top [0])
        query += " and z.longitude >= " + str(left[1])
        query += " and z.longitude <= " + str(right[1])
        query += " order by 1 "

        cursor.execute(query)

        connection.rollback()

        rows = cursor.fetchall()
        if rows != []:
            for row in rows:
                zips_1mi.append(row[0])
                lat_1mi.append(row[1])
                lon_1mi.append(row[2])

In [50]:
# creating the customers density heat map
connection.rollback()

query = """

drop table if exists zips_1mi;

"""

cursor.execute(query)

connection.commit()

connection.rollback()

query = """

CREATE TABLE zips_1mi(
    zips varchar,
    latitude varchar,
    longitude varchar
)

"""

cursor.execute(query)

connection.commit()

for i in range(len(zips_1mi)):
    connection.rollback()
    cursor.execute("insert into zips_1mi (zips, latitude, longitude) values (%s,%s,%s);", 
                   (zips_1mi[i],lat_1mi[i],lon_1mi[i]))
    connection.commit()

connection.rollback()

rollback_before_flag = True
rollback_after_flag = True


query = """

select z.latitude, z.longitude
from customers as c
join zips_1mi as z on c.zip=z.zips

"""

locats = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

locats


,latitude,longitude
0,37.7797,-122.4192
1,37.7797,-122.4192
2,37.7797,-122.4192
3,37.7797,-122.4192
4,37.7797,-122.4192
...,...,...
6740,37.8088,-122.2691
6741,37.8088,-122.2691
6742,37.8088,-122.2691
6743,37.8088,-122.2691


In [51]:
# map marker of BART stations

rollback_before_flag = True
rollback_after_flag = True


query = """

select station, latitude, longitude
from stations

"""

station_gps = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
station_gps

,station,latitude,longitude
0,12th Street,37.803608,-122.272006
1,16th Street Mission,37.764847,-122.420042
2,19th Street,37.807869,-122.268980
3,24th Street Mission,37.752000,-122.418700
4,Antioch,37.996281,-121.783404
5,Ashby,37.853068,-122.269957
6,Balboa Park,37.721667,-122.447500
7,Bay Fair,37.697000,-122.126500
8,Berryessa,37.368361,-121.874655
9,Castro Valley,37.690748,-122.075679


In [52]:
# map of the Bart station and 1 mile radius. around the pick up stations and the Berekeley store.

cust_map = folium.Map(location = [lat,lon], zoom_start=9)
heat = HeatMap(
    list(zip(locats.latitude.values, locats.longitude.values)),
    radius=12
)

for i in range(len(station_gps)):
    folium.Marker(
        location = [station_gps.iloc[i]['latitude'], station_gps.iloc[i]["longitude"]],
        popup=station_gps.iloc[i]["station"],
        icon = folium.DivIcon(html="""
            <div><svg>
                <circle cx="5" cy="5" r="5" fill="#170918" opacity="1"/>
            </svg></div>
        """)
    ).add_to(cust_map)

cust_map.add_child(heat)
